In [1]:
import nltk
from nltk.corpus import wordnet as wn
import itertools
from typing import Callable, Tuple
from nltk.corpus.reader.wordnet import Synset
import unittest

In [2]:
# Based on alexis's answer to http://stackoverflow.com/questions/30829382/check-the-similarity-between-two-words-with-nltk-with-python


def wn_sim(w1: str, w2: str, metric: Callable[[Synset, Synset], float] = wn.wup_similarity) -> Tuple[float, Synset, Synset]:
    """
    Find the pair of wordnet synsets corresponding to the given words that are most similar
    according to the given metric. The result is the similarity followed--in order--by the
    synsets chosen for w1 and w2.
    """
    syns1 = wn.synsets(w1)
    syns2 = wn.synsets(w2)
    best = max((metric(s1, s2) or 0, s1, s2) for s1, s2 in itertools.product(syns1, syns2))
    
    return best

def wn_exists(word: str) -> bool:
    """
    Returns true if the word exists according to wordnet.
    """
    return len(wn.synsets(word)) > 0

class TestWNUtils(unittest.TestCase):
    def test_wn_sim(self):
        self.assertEqual(wn_sim('dog', 'cat', wn.wup_similarity), 
                         (0.8571428571428571, wn.synset('dog.n.01'), wn.synset('cat.n.01')))
        self.assertEqual(wn_sim('dog', 'cat'), 
                         (0.8571428571428571, wn.synset('dog.n.01'), wn.synset('cat.n.01')))
        
    def test_wn_exists(self):
        self.assertTrue(wn_exists('dog'))
        self.assertTrue(wn_exists('flagging'))
        self.assertFalse(wn_exists('plnsgi'))


suite = unittest.TestLoader().loadTestsFromTestCase(TestWNUtils)
unittest.TextTestRunner(verbosity=2).run(suite)

test_wn_exists (__main__.TestWNUtils) ... ok
test_wn_sim (__main__.TestWNUtils) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.133s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [3]:
import random
from typing import List, Tuple, TypeVar

# From https://en.wikipedia.org/wiki/Letter_frequency#Relative_frequencies_of_letters_in_the_English_language
letter_freqs = [
    ('a', 8.167),
    ('b', 1.492),
    ('c', 2.782),
    ('d', 4.253),
    ('e', 12.702),
    ('f', 2.228),
    ('g', 2.015),
    ('h', 6.094),
    ('i', 6.966),
    ('j', 0.153),
    ('k', 0.772),
    ('l', 4.025),
    ('m', 2.406),
    ('n', 6.749),
    ('o', 7.507),
    ('p', 1.929),
    ('q', 0.095),
    ('r', 5.987),
    ('s', 6.327),
    ('t', 9.056),
    ('u', 2.758),
    ('v', 0.978),
    ('w', 2.360),
    ('x', 0.150),
    ('y', 1.974),
    ('z', 0.074)]

T = TypeVar('T')
def select_by_prob(choices: List[Tuple[T, float]]) -> T:
    """
    Choose one of the choices, respecting their frequencies (as percentages, i.e., intended to total to 100).
    
    Note: choices must be non-empty, and the frequency of the last entry is ignored and assumed to be 100 minus
    the sum of the remaining entries.
    
    TODO: this would be more efficient using binary search if we made the probabilities into cumulative probabilities instead.
    (Or, if needed, we could do this as a sort of Trie for faster performance.)
    """
    num = random.random() * 100
    mass_passed = choices[0][1]
    index = 0
    while index < len(choices) - 1 and mass_passed < num:
        index += 1
        mass_passed += choices[index][1]
        
    return choices[index][0]

def select_letter() -> str:
    """
    Select a letter respecting English letter frequencies.
    """
    return select_by_prob(letter_freqs)

In [4]:
# TESTING CODE for select_letter
counts = {}
for c in (select_letter() for i in range(10000)):
    if c not in counts:
        counts[c] = 0
    counts[c] += 1

total = sum(counts.values())
for k in counts.keys():
    counts[k] = 100.0 * counts[k] / total

for (c, freq) in letter_freqs:
    if c in counts:
        counts[c] -= freq

error = sum(counts.values())

# error should be small!
error

0.0009999999999988907

In [5]:
from ipywidgets import widgets
from IPython.display import display

def play(target_word):
    """
    Gives a list of available letters and scores the
    constructed word for how similar it is to the target word.
    """
    letters = [select_letter().upper() for i in range(15)]
    used_letters = []
    unused_letters = letters[:]
    
    layout = widgets.HBox()
    in_text = widgets.Text()
    letters_text = widgets.HTML(disabled=True, value=' '.join(unused_letters))
    valid = widgets.Valid(
        value=wn_exists(''),
    )
    layout.children = [in_text, valid]
    
    def handle_submit(sender):
        if valid.value:
            print(wn_sim(in_text.value, target_word))
    
    def handle_trait_change(name, new):
        assert(name == 'value')
        
        result = True
        used_letters[:] = new
        unused_letters[:] = letters[:]
        for c in used_letters:
            c = c.upper()
            if c in unused_letters:
                unused_letters.remove(c)
            else:
                result = False
        
        letters_text.value = ' '.join(unused_letters)
        valid.value = result and wn_exists(new)
    
    in_text.on_trait_change(handle_trait_change, 'value')
    in_text.on_submit(handle_submit)
    display(layout)
    display(letters_text)

In [6]:
play('attack')

(0.8333333333333334, Synset('call_on_the_carpet.v.01'), Synset('attack.v.02'))
(0.7, Synset('flood.n.05'), Synset('attack.n.06'))
(0.7, Synset('beat.n.09'), Synset('attack.n.06'))
(0.3333333333333333, Synset('noodle.n.01'), Synset('attack.n.08'))
(0.3333333333333333, Synset('tar.v.01'), Synset('assail.v.01'))
(0.3333333333333333, Synset('bean.v.01'), Synset('assail.v.01'))
(0.6666666666666666, Synset('deal.n.08'), Synset('attack.n.02'))
(0.3333333333333333, Synset('human_body.n.01'), Synset('attack.n.08'))
(0.26666666666666666, Synset('ten.n.01'), Synset('fire.n.09'))
(0.7, Synset('tear.n.04'), Synset('attack.n.06'))
